# DNN (Deep Neural Network) :

Epoch 1/50
6112/6112 [==============================] - 41s 6ms/step - loss: 0.0605 - accuracy: 0.9852 - val_loss: 0.0331 - val_accuracy: 0.9896
Epoch 2/50
6112/6112 [==============================] - 49s 8ms/step - loss: 0.0365 - accuracy: 0.9894 - val_loss: 0.0306 - val_accuracy: 0.9902
Epoch 3/50
6112/6112 [==============================] - 48s 8ms/step - loss: 0.0354 - accuracy: 0.9893 - val_loss: 0.0312 - val_accuracy: 0.9900
Epoch 4/50
6112/6112 [==============================] - 41s 7ms/step - loss: 0.0337 - accuracy: 0.9899 - val_loss: 0.0297 - val_accuracy: 0.9909
Epoch 5/50
6112/6112 [==============================] - 54s 9ms/step - loss: 0.0326 - accuracy: 0.9902 - val_loss: 0.0306 - val_accuracy: 0.9905
Epoch 6/50
6112/6112 [==============================] - 55s 9ms/step - loss: 0.0326 - accuracy: 0.9901 - val_loss: 0.0306 - val_accuracy: 0.9901
Epoch 7/50
6112/6112 [==============================] - 55s 9ms/step - loss: 0.0317 - accuracy: 0.9903 - val_loss: 0.0288 - val_ac

2038/2038 [==============================] - 3s 2ms/step
Accuracy: 0.9914860324911409
Precision: 0.8608582574772432
Recall: 0.7951951951951952
F1 Score: 0.8267249453637217
auc-Score: 0.90


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

# Chargement du dataset
df = pd.read_csv('../final_desequilibre.csv')

# Séparation des features (X) et de la variable cible (y)
X = df.drop('isFraud', axis=1)
y = df['isFraud']

# Normalisation des données
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Division des données en ensembles d'entraînement, de validation et de test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_validation, X_test, y_validation, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Créez le modèle DNN
model = models.Sequential()

# Couches fully connected (densément connectées) avec Dropout
model.add(layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.5))

# Couche de sortie
model.add(layers.Dense(1, activation='sigmoid'))  # Une seule sortie binaire (fraude ou non)

# Compilation du modèle
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Binary cross-entropy pour une classification binaire
              metrics=['accuracy'])

# Entraînez le modèle avec plus d'epochs
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_validation, y_validation))

# Visualisation de l'historique d'entraînement
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss vs. Epochs')
plt.show()


# Visualisation de l'historique d'entraînement
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Train Accuracy')  # Utilisez 'accuracy' ou 'acc' selon votre historique
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')  # Utilisez 'val_accuracy' ou 'val_acc'
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy vs. Epochs')
plt.show()

# Prédictions sur l'ensemble de test avec un seuil de décision de 0.3
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.3)

# Calcul des métriques
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"auc-Score: {auc:.2f}")


In [2]:

# Calcul des métriques
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)


dnn_results = pd.DataFrame([['Deep neurual network', acc,prec,rec, f1, auc]],
               columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score', 'AUC'])

# Sauvegardez le DataFrame dans un fichier CSV
dnn_results.to_csv('../evaluations/dnn_results.csv', index=False)

dnn_results

,Model,Accuracy,Precision,Recall,F1 Score,AUC
0,Deep neurual network,0.991486,0.860858,0.795195,0.826725,0.895913


In [3]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
# Affichage de la matrice de confusion
confusion = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(confusion, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()